Ensure you are using the preinstalled Jupyter kernel `amcinsights`

# Amazon Ads Reporting
Authenticate with Amazon Ads Auth before using

#### -----------------------
## Global Configs
Run this cell to initialize the notebook interface

In [ ]:
import json
import re
import boto3
import os
from dotenv import load_dotenv
load_dotenv()

boto3_session = boto3.Session()
lambda_client = boto3_session.client('lambda')

invoke_reporting_lambda_name = os.environ['INVOKE_ADS_REPORT_SM_NAME']
get_profiles_lambda_name = os.environ['GET_PROFILES_NAME']
schedule_ads_report_lambda_name = os.environ['SCHEDULE_ADS_REPORT_NAME']

In [ ]:
# (Optional) If multi-credential secret configured, Auth Id to use for API requests.
# If you did not configure your secret for multiple credentials, leave as None.
auth_id = None

#### -----------------------
## Profiles
Run this cell to return all your connected profiles in the specified regions

In [ ]:
region_list = ["North America", "Europe", "APAC"]

In [ ]:
get_profiles_request = {
    "region": region_list,
    "authId": auth_id
}

#############################################
response = lambda_client.invoke(
    FunctionName=get_profiles_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(get_profiles_request).encode('UTF-8')
)

# Print output
payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
print(json.dumps(payload_json, indent=4))
 

#### -----------------------
## Reporting
The following cells show how to request a report from the Amazon Ads Reporting API. 
<br />
Refer to the [API Docs](https://advertising.amazon.com/API/docs/en-us/offline-report-prod-3p#tag/Asynchronous-Reports/operation/createAsyncReport) or [Developer Guide](https://advertising.amazon.com/API/docs/en-us/guides/reporting/overview) for additional information on how to structure your requests.
<br />
<br />
To start, initialize the cell below with the `profile_id` and `region` you want to submit the report request to.
<br />
<br />
There is also an optional parameter `table_name` that can be included. Left out, the data lake will default to using the provided {Profile Id}-{Report Type Id} as the destination table.

In [ ]:
# REQUIRED
profile_id = ""
region = "North America"

# OPTIONAL
table_name = "TestTable"

### Create Report Request


In [ ]:
# REQUIRED
request_body = {
    "startDate": "TODAY(-1)",
    "endDate": "TODAY(-1)",
    "configuration": {
        "adProduct": "SPONSORED_PRODUCTS",
        "groupBy": ["asin"],
        "columns": [
            "purchasedAsin", 
            "advertisedAsin", 
            "adGroupName", 
            "campaignName", 
            "sales14d", 
            "campaignId",
            "adGroupId", 
            "keywordId", 
            "keywordType", 
            "keyword"
        ],
        "reportTypeId": "spPurchasedProduct",
        "timeUnit": "SUMMARY"
    }
}

In [ ]:
create_report_request = {
    "profileId": profile_id,
    "region": region,
    "requestBody": request_body,
    "authId": auth_id,
    "tableName": table_name
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_reporting_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_report_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

### Schedule Report Request

CRON format: https://docs.aws.amazon.com/scheduler/latest/UserGuide/schedule-types.html#cron-based

In [ ]:
# REQUIRED
report_request = {
    "requestBody": {
        "startDate": "TODAY(-6)",
        "endDate": "TODAY()",
        "configuration": {
            "adProduct": "SPONSORED_PRODUCTS",
            "groupBy": ["asin"],
            "columns": [
                "purchasedAsin", 
                "advertisedAsin", 
                "adGroupName", 
                "campaignName", 
                "sales14d", 
                "campaignId",
                "adGroupId", 
                "keywordId", 
                "keywordType", 
                "keyword"
            ],
            "reportTypeId": "spPurchasedProduct",
            "timeUnit": "SUMMARY"
        }
    },
    "tableName": table_name,
    "profileId": profile_id,
    "region": region,
    "authId": auth_id
}
schedule_expression = "cron(0 23 ? * 7 *)"
rule_name = "testrule"

# OPTIONAL
rule_description = "Trigger spPurchasedProduct every Sunday at 11pm"

In [ ]:
create_schedule_request = {
    "report_request": report_request,
    "schedule_expression": schedule_expression,
    "rule_name": rule_name,
    "rule_description": rule_description
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=schedule_ads_report_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_schedule_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    response_message = response.get('Payload').read().decode('UTF-8')
    response_message = response_message.strip('"')
    print(f"Event Rule Url: {response_message}")

----
Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0